---
title: Análise dos Lotes - Lotes para Uso
author: Geraldo Braz Junior
description: Identificar lotes que poderiam ser usados para habitações e/ou para equipamentos públicos.
show-code: False
params:
    ocupacao_btn:
        label: Ocupação
        input: select
        value: ['Edificação Subutilizada', 'Estacionamento', 'Edificação sem uso', 'Terreno não edificado', 'ND']
        choices: ['Edificação Subutilizada', 'Estacionamento', 'Edificação sem uso', 'Terreno não edificado', 'ND']
        multi: True
    conservacao_btn:
        label: Conservação
        input: select
        value: ['Regular', 'Ótimo', 'Bom', 'Péssimo ou em Risco', 'Perda', 'ND']
        choices: ['Regular', 'Ótimo', 'Bom', 'Péssimo ou em Risco', 'Perda', 'ND']
        multi: True
    tipologia_btn:
        label: Tipologia
        input: select
        value: ['SOBRADO', 'SOBRADO COLONIAL', 'CASA', 'ND']
        choices: ['APTO FRENTE', 'APTO FUNDOS', 'APTO TERREO', 'BARRACO', 'Casa', 'CASA', 'COB METALICA', 'COM C/ RESIDENCIA', 'DEPOSITO', 'EDIFICAÇÃO COMPLEMENTAR', 'GALPAO', 'GARAGEM', 'Loja/sala/conjunto', 'LOJA/SALA/CONJUNTO', 'PREDIAL', 'SOBRADO', 'SOBRADO COLONIAL', 'TELHEIRO', 'TEMPLO', 'undefined', 'ND']
        multi: True    
    area_btn:
        label: Área
        input: range
        value: [200, 1000]
        min: 0
        max: 1000
    distancia_btn:
        label: Distância Linha de Ônibus
        input: text    
        rows: 1    
    divida_btn:
        label: Lotes com Dívida
        input: checkbox    
        value: False       
    indicador_btn:
        label: Indicadores
        input: select
        value: 'Selecione'
        choices: ['Selecione', 'População Total (densidade)', 'Esgotamento sanitário via fossa séptica', 'Esgotamento sanitário via fossa rudimentar', 'Esgotamento sanitário via vala', 'Esgotamento sanitário via rio, lago ou mar', 'Esgotamento sanitário via outro escoadouro']
        multi: False    
    equipamento_btn:
        label: Equipamento
        input: select
        value: 'Educação'
        choices: ['Assistência Social', 'Educação', 'Equipamento Cultural', 'Saúde', 'Segurança']
        multi: False    
---

In [4]:
#usando a API para obter e manipular os dados

import os
import sys

#path para a biblioteca do apiModulo. Ajuste de acordo com a necessidade
sys.path.insert(0, os.path.abspath('../'))
from apiModulo.api import *

In [5]:
#obtendo lotes
lotes = obterCamada('lotes')

#obtendo lotes por ponto
lotes['centroid'] =  lotes['geometria'].centroid

#obtendo zonas de interesse social
zis = obterCamada('zis')

#obtendo linhas de onibus
linhas = obterCamada('linhas_onibus')

#obtendo tombamento estadual
tomb_estadual = obterCamada('tombamento_estadual')

#obtendo tombamento federal
tomb_federak = obterCamada('tombamento_federal')

#obtendo hex de densidade populacional
hdx = obterCamada('hdx_populacao_2020')

#adicionando equipamentos informados pela prefeitura
equipamentos = obterCamada('equipamento_publico', simples=False)

#adicionando dados do ibge, em setor censitário, quanto a domicilio01, v018-v022 - esgotamento
domicilios, m = obterIndicador(tabela='ibge_domicilio01', definicao="""'v018', 'v019', 'v020', 'v021', 'v022'""")
domicilios = limparDados(domicilios)


In [6]:
#filtrando dados que existem por ocupação
#ocupacao = lotes[~lotes['ocupacao'].isnull()]
ocupacao = lotes

ocupacao['ocupacao'].fillna(value='ND', inplace=True)
ocupacao['conservacao'].fillna(value='ND', inplace=True)
ocupacao['tipologia_semfaz'].fillna(value='ND', inplace=True)

#print(ocupacao.shape[0])

In [7]:
#estado inicial das variáveis de seleção
ocupacao_btn = ['Edificação sem uso', 'Terreno não edificado']
conservacao_btn = ['Regular', 'Ótimo', 'Bom', 'Péssimo ou em Risco', 'Perda', 'ND']
tipologia_btn = ['SOBRADO', 'SOBRADO COLONIAL', 'CASA', 'ND']
area_btn = [200, 1000]
equipamento_btn = 'Educação'
indicador_btn = 'Selecione'
distancia_btn = '400'
divida_btn = True

#indicador_btn = 'População Total (densidade)'

In [9]:
import folium

indicador_var = ['', 'População Total (densidade)', 'ibge_domicilio01_v018', 'ibge_domicilio01_v019', 'ibge_domicilio01_v019', 'ibge_domicilio01_v020', 'ibge_domicilio01_v021', 'ibge_domicilio01_v022']
alias_var = ['Selecione', 'População Total (densidade)', 'Esgotamento sanitário via fossa séptica', 'Esgotamento sanitário via fossa rudimentar', 'Esgotamento sanitário via vala', 'Esgotamento sanitário via rio, lago ou mar', 'Esgotamento sanitário via outro escoadouro']
tipo_btn = 'Calor'

#ação quando selecionar itens
filtrado = ocupacao[(ocupacao['AREA_HTL']>=area_btn[0]) & (ocupacao['AREA_HTL']<=area_btn[1])]
filtrado = filtrado[filtrado['ocupacao'].isin(ocupacao_btn)]
filtrado = filtrado[filtrado['conservacao'].isin(conservacao_btn)]
filtrado = filtrado[filtrado['tipologia_semfaz'].isin(tipologia_btn)]


equip_filtrado = equipamentos[ (equipamentos['tipo'] == equipamento_btn) ]
        
m = visMultiMapa(width='90%', MAPA_ZOOM=15)

#filtro de distancia
#if distancia_btn != '': 
#    linhas_buffer = linhas.copy()
#    print(filtrado.shape)
#    filtrado = filtrado [ filtrado.intersects(linhas_buffer.buffer(0.0008)) ]
#    print(filtrado.shape)
    
#plot dos lotes
m = visMultiMapa(m, tipo='layer', 
            dado=filtrado,                         
            variavel='ocupacao',
            alias=f"Lotes com  Ocupação: {ocupacao_btn}, área mínima: {area_btn} e estado de conservação: {conservacao_btn}" )   

#filtro de dívida
if divida_btn:
    divida = lotes[~lotes['divida_ipt'].isnull()]
    divida = divida[(divida['divida_ipt']!='s/ divida') & (divida['divida_ipt']!='s/divida') ]
    divida['geometria'] = divida['geometria'].centroid
    style_function = lambda x: {'fillColor': '#C0392B', "color": "#641E16"}
    m = visMultiMapa(m, tipo='marcador', dado=divida, variavel='divida_ipt', alias=f"Dívida" )   

if tipo_btn == 'Calor':
    m = visMultiMapa(m, tipo='calor', dado=equip_filtrado, alias=equipamento_btn, variavel='tipo')     
else:
    m = visMultiMapa(m, tipo='marcador', dado=equip_filtrado, alias=equipamento_btn, variavel='tipo')             

style_function = lambda x: {'fillColor': '#EB984E', "color": "#ff7800"}
m = visMultiMapa(m, tipo='layer', alias='Zona de Interesse Social', dado = zis, style=style_function)

style_function = lambda x: {'fillColor': '#8E44AD', "color": "#8E44AD"}
m = visMultiMapa(m, tipo='layer', alias='Linhas de ônibus', dado = linhas, style=style_function)

style_function = lambda x: {'fillColor': '#FDFEFE', "color": "#52BE80"}
m = visMultiMapa(m, tipo='layer', alias='Limites Tombamento Estadual', dado = tomb_estadual, style=style_function)

style_function = lambda x: {'fillColor': '#FDFEFE', "color": "#CD6155"}
m = visMultiMapa(m, tipo='layer', alias='Limites Tombamento Federal', dado = tomb_federak, style=style_function)

if indicador_btn != 'Selecione':
    index = alias_var.index(indicador_btn)
    if index == 1:
        m = visMultiMapa(m, tipo='choro', dado=hdx, variavel='population_2020', alias=alias_var[index]) 
    else:
        m = visMultiMapa(m, tipo='choro', dado=domicilios, variavel=indicador_var[index], alias=alias_var[index]) 

folium.LayerControl().add_to(m)
m
